In [4]:
#install libraries
# !pip install dash==1.13.3

### Imports

In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components
from datetime import date, timedelta

import os

## Read all files an concat them into a big dataframe

In [8]:
entidad = 2 #Baja California
municipio = 2 #Mexicali
resultado = 1 #Positivo

files = os.listdir('data/')
csv_files = [ filename for filename in files if filename.endswith( '.csv' ) ]
csv_files.sort()

df_master = pd.concat([pd.read_csv('data/'+f, encoding = 'ISO-8859-1') for f in csv_files ])
df_master.to_csv('data/200629-200726COVID19MEXICO.csv', index=False, encoding='ISO-8859-1')



## Day by Day Delta

In [11]:
entidad = 2 #Baja California
municipio = 2 #Mexicali
resultado = 1 #Positivo

df_master = pd.read_csv('./data/200629-200726COVID19MEXICO.csv', encoding = "ISO-8859-1")
df_master = df_master.loc[df_master['ENTIDAD_RES'] == 2] #entidad = Baja California
df_master = df_master.loc[df_master['MUNICIPIO_RES'] == 2] # municipio = mexicali
# df_master = df_master.loc[df_master['RESULTADO'] == 1] # resultado = positivo SARS-COV-2

df_master.loc[df_master['RESULTADO'] == 1, 'CONFIRMADO'] = ['1']
df_master.loc[df_master['RESULTADO'] == 3, 'SOSPECHOSO'] = ['1']
df_master.loc[df_master['RESULTADO'] == 2, 'NEGATIVO'] = ['1']
df_master = df_master[['FECHA_ACTUALIZACION', 'CONFIRMADO', 'SOSPECHOSO', 'NEGATIVO']]

df_master_by_date = df_master.groupby("FECHA_ACTUALIZACION",as_index=False).count()
df_master_by_date
df_master_by_date.to_csv('summaries/200629-200726COVID19MEXICO.csv', index=False, encoding='ISO-8859-1')






### Confirmados Mexicali por Fecha de Inicio de Sintomas

In [7]:
df_src = pd.read_csv('./newest/201011COVID19MEXICO.csv', encoding = "ISO-8859-1")

#confirmados
df = df_src.loc[df_src['ENTIDAD_RES'] == 2] #entidad = Baja California
df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali
df = df.loc[df['RESULTADO_LAB'] == 1] # resultado = positivo SARS-COV-2
df_confirmed = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df_confirmed_by_date = df_confirmed.groupby("FECHA_SINTOMAS",as_index=False).count()

df_confirmed_by_date.columns = ['FECHA_SINTOMAS','CONFIRMADOS']

fig = px.bar(df_confirmed_by_date,df_confirmed_by_date.FECHA_SINTOMAS,df_confirmed_by_date.CONFIRMADOS,
            title="Confirmados Mexicali por fecha inicio de sintomas");
fig.show()






### Sospechosos Mexicali por Fecha Inicio de Sintomas

In [8]:
#sospechosos
df = df_src.loc[df_src['ENTIDAD_RES'] == 2] #entidad = Baja California
df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali
df = df.loc[df['RESULTADO_LAB'] == 3] # resultado = sospechoso
df_suspected = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df_suspected_by_date = df_suspected.groupby("FECHA_SINTOMAS",as_index=False).count()

df_suspected_by_date.columns = ['FECHA_SINTOMAS','SOSPECHOSOS']

fig = px.bar(df_suspected_by_date,df_suspected_by_date.FECHA_SINTOMAS,df_suspected_by_date.SOSPECHOSOS);
fig.show()


### Confirmados + Sospechosos por fecha de Inicio de Sintomas

In [10]:
df_src = pd.read_csv('./newest/201011COVID19MEXICO.csv', encoding = "ISO-8859-1")

df = df_src.loc[df_src['ENTIDAD_RES'] == 2] #entidad = Baja California
df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali
df = df.loc[(df['RESULTADO_LAB'] == 1) | (df['RESULTADO_LAB'] == 3)] # resultado = positivo SARS-COV-2 o Pendiente
df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']
df.loc[df['RESULTADO_LAB'] == 3, 'SOSPECHOSO'] = ['1']
df_by_date = df.groupby("FECHA_SINTOMAS",as_index=False).count()
df_by_date

fig = go.Figure(data=[
    go.Bar(name='Confirmados', x=df_by_date.FECHA_SINTOMAS, y=df_by_date.CONFIRMADO),
    go.Bar(name='Sospechosos', x=df_by_date.FECHA_SINTOMAS, y=df_by_date.SOSPECHOSO)
])
fig.update_layout(
    title="Curva Epidemica para Mexicali, BC",
    legend_title="Tipo de Resultado"
)

fig.update_layout(barmode='stack')
fig.show()

## Casos Estimados Mexicali

In [11]:
df_src = pd.read_csv('./newest/201011COVID19MEXICO.csv', encoding = "ISO-8859-1")

df = df_src.loc[df_src['ENTIDAD_RES'] == 2] #entidad = Baja California
df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali
df = df.loc[(df['RESULTADO_LAB'] == 1) | (df['RESULTADO_LAB'] == 3)] # resultado = positivo SARS-COV-2 o Pendiente
df = df[['FECHA_SINTOMAS','RESULTADO_LAB']]

df.loc[df['RESULTADO_LAB'] == 1, 'CONFIRMADO'] = ['1']
df.loc[df['RESULTADO_LAB'] == 3, 'SOSPECHOSO'] = ['1']
df_by_date = df.groupby("FECHA_SINTOMAS",as_index=False).count()
df_by_date["SOSPECHOSO"] = pd.to_numeric(df_by_date["SOSPECHOSO"])
df_by_date["CONFIRMADO"] = pd.to_numeric(df_by_date["CONFIRMADO"])
# df['SOSPECHOSO'] = df['SOSPECHOSO'].astype(int)

df_by_date['ESTIMADO'] = df_by_date['CONFIRMADO'] + (df_by_date['SOSPECHOSO']/2)
df_by_date

fig = px.line(df_by_date, df_by_date.FECHA_SINTOMAS, df_by_date.ESTIMADO, 
              title='Estimados Mexicali por fecha inicio de sintomas')
fig.show()


### Confirmados Activos en Mexicali

In [6]:
fourteen_days_ago = date.today() - timedelta(days=15)
fourteen_days_ago = fourteen_days_ago.strftime("%Y-%m-%d");
fourteen_days_ago



df = df_src.loc[df_src['ENTIDAD_RES'] == 2] #entidad = Baja California
df = df.loc[df['MUNICIPIO_RES'] == 2] # municipio = mexicali
df = df.loc[df['RESULTADO'] == 1] # resultado = confirmado
df_confirmed = df[['FECHA_SINTOMAS','RESULTADO']]
df_confirmed_active = df_confirmed.loc[df_confirmed['FECHA_SINTOMAS'] >= fourteen_days_ago]
df_confirmed_active_by_date = df_confirmed_active.groupby("FECHA_SINTOMAS",as_index=False).count()
df_confirmed_active_by_date = df_confirmed_active_by_date.sum(axis = 0, skipna = True) 
df_confirmed_active_by_date['RESULTADO']



# date_time = now.strftime("%m-%d-%Y, %H:%M:%S")
# print("date and time:",date_time)	

# df_suspected_by_date = df_suspected.groupby("FECHA_SINTOMAS",as_index=False).count()

# df_suspected_by_date.columns = ['FECHA_SINTOMAS','SOSPECHOSOS']




158

## Dash Testing

In [130]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "wide-form" data frame with no index
# see https://plotly.com/python/wide-form/ for more options
df = pd.DataFrame({"x": [1, 2, 3], "SF": [4, 1, 2], "Montreal": [2, 4, 5]})

fig = px.bar(df, x="x", y=["SF", "Montreal"], barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of previous arguments ['x'] is 3